In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from pprint import pprint
import winsound
import time

In [2]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

11:32:47


In [3]:
df = pd.read_csv('./../new_DMThotel/data/cleaned_data_dates.csv', index_col=0)
print(df.shape)
print(df.columns)
df.head()

(276780, 33)
Index(['srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'click_bool', 'booking_bool', 'number_of_comp', 'aggr_comp_rate_dif',
       'aggr_comp_availability', 'aggr_comp_price_dif', 'year', 'month',
       'hour', 'time_of_day'],
      dtype='object')


,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,click_bool,booking_bool,number_of_comp,aggr_comp_rate_dif,aggr_comp_availability,aggr_comp_price_dif,year,month,hour,time_of_day
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,154578,5,219,100,120561,2,4.5,1,2.08,0.045390,...,0,0,1,0.0,0.0,0.0,2013,5,6,1
1,71428,32,220,220,105534,5,4.0,1,5.16,0.569900,...,0,0,4,0.0,0.0,0.0,2012,11,9,1
2,157079,12,99,99,77267,4,4.5,1,1.79,0.666700,...,0,0,2,1.0,0.0,2.0,2012,11,12,2
3,1153,5,219,219,77168,3,3.0,1,1.79,0.034095,...,0,0,0,0.0,1.0,0.0,2013,2,15,2
4,170136,5,88,88,80780,4,4.0,1,2.71,0.010900,...,0,0,0,0.0,0.0,0.0,2012,11,12,2


In [4]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

11:32:58


In [5]:
data = df[['visitor_location_country_id', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'number_of_comp', 'aggr_comp_rate_dif',
       'aggr_comp_availability', 'aggr_comp_price_dif', 'year', 'month',
       'hour', 'time_of_day']]
y = df['click_bool']
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2)

In [6]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

11:32:59


In [7]:
rf = RandomForestClassifier(random_state = 42)
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

# n_estimators = number of trees in the foreset
# max_features = max number of features considered for splitting a node
# max_depth = max number of levels in each decision tree
# min_samples_split = min number of data points placed in a node before the node is split
# min_samples_leaf = min number of data points allowed in a leaf node
# bootstrap = method for sampling data points (with or without replacement)

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [8]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

11:32:59


In [9]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [10]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

11:32:59


In [11]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed: 27.0min remaining: 23.7min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 47.4min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [12]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

12:34:43


In [13]:
rf_random.best_params_

{'n_estimators': 1400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [17]:
with open('./best_params.txt', 'a') as outfile:
    for key, value in rf_random.best_params_.items():
        outfile.write(f'{key}: {value}, ')

In [ ]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
predictions = base_model.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
best_random = rf_random.best_estimator_
predictions = best_random.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

In [23]:
frequency = 1700  # Set Frequency To 2500 Hertz
duration = 500  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)